In [ ]:
pip install kaggle

In [1]:
!mkdir -p ~/.kaggle

In [2]:
#Importing the kaggle dataset
#Uploading the kaggle.json file
#Setting up the kaggle api

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
#Importing the Twitter Sentiment Dataset
# API to fetch the dataset from kaggle.
# !kaggle datasets download -d kazanova/sentiment140
!kaggle datasets download -d yasserh/imdb-movie-ratings-sentiment-analysis
#!mark is used to run shell commands

401 - Unauthorized - Unauthenticated


In [4]:
#extracting the zip file to directory, remember that
#csv already exists inside zip file.
from zipfile import ZipFile
data = '/content/imdb-movie-ratings-sentiment-analysis.zip'

with ZipFile(data,'r') as zip:
  zip.extractall()
  print('The dataset is extracted.')


FileNotFoundError: [Errno 2] No such file or directory: '/content/imdb-movie-ratings-sentiment-analysis.zip'

Importing and installing the dependencies.

In [ ]:
#Importing and installing the dependencies.
import numpy as np
import pandas as pd
#used to create dataframes, structured table using pandas
import re
#Used for pattern matching re library
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#Used for stemming the words and nltk library is for nlp.
from sklearn.feature_extraction.text import TfidfVectorizer
#Used to convert textual data to numertical data Vectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#We are using logistic regression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))
#Stopwords are the words which don't have any influential meaning.
#These words are not required for out processing.
#Removing the stopwords as they are non-influential.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Processing

In [ ]:
#Data Processing
#loading the data from csv file to pandas dataframe.
t_data = pd.read_csv('/content/movie.csv', encoding ='ISO-8859-1')


In [ ]:
#Checking the number of rows and coloumns in t_data
t_data.shape

#printing the first five rows of the dataframe.
t_data.head()
t_data = t_data.rename(columns ={'ï»¿text':'text'})
print(t_data.head())


                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not know what a particula...      0
3  Even though I have great interest in Biblical ...      0
4  Im a die hard Dads Army fan and nothing will e...      1


In [ ]:
#As it is not reading the coloumn names we are naming the coloumns.
#as first data point is considered as coloumn name.
#Naming the coloumn name in t_data.
column_names =['text','id']
t_data = pd.read_csv('/content/movie.csv', names = column_names, encoding ='ISO-8859-1')

In [ ]:
t_data.shape

(40001, 2)

In [ ]:
t_data.shape

t_data.head()

,text,id
0,ï»¿text,label
1,I grew up (b. 1965) watching and loving the Th...,0
2,"When I put this movie in my DVD player, and sa...",0
3,Why do people who do not know what a particula...,0
4,Even though I have great interest in Biblical ...,0


In [ ]:
t_data.drop(t_data.index[0])

,text,id
1,I grew up (b. 1965) watching and loving the Th...,0
2,"When I put this movie in my DVD player, and sa...",0
3,Why do people who do not know what a particula...,0
4,Even though I have great interest in Biblical ...,0
5,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39996,"""Western Union"" is something of a forgotten cl...",1
39997,This movie is an incredible piece of work. It ...,1
39998,My wife and I watched this movie because we pl...,0
39999,"When I first watched Flatliners, I was amazed....",1


In [ ]:
#Read the last 5 tweets as opposite to head
t_data.tail()

,text,id
39996,"""Western Union"" is something of a forgotten cl...",1
39997,This movie is an incredible piece of work. It ...,1
39998,My wife and I watched this movie because we pl...,0
39999,"When I first watched Flatliners, I was amazed....",1
40000,"Why would this film be so good, but only gross...",1


In [ ]:
#Dealing with missing values, by replacing or dropping
#Checking the values and counting the number of missing values.
t_data.isnull().sum()

text    0
id      0
dtype: int64

In [ ]:
#Understanding the distribution of the target variable.
#Checking the distribution of target columns
#Here the dataset is evenly distributed but if not we have to
#perform upsampling or downsampling.
t_data['id'].value_counts()

0        20019
1        19981
label        1
Name: id, dtype: int64

In [ ]:
#Convert target columns value 4 to 1
# inplace means this change should take place in original dataset
#Now 0 is negative tweet and 1 is positive tweet.
# t_data.replace({'target':{4:1}}, inplace = True)
# But I think doing this will bias the model and hence affect accuracy.

In [ ]:
#Stemming
#It is the process that is done to reduce the word to its root word (keyword).
# ex. actor, actress, acting.
#solution: act
#ex. choco, chocolatey, chocolates
#solution: chocolate.
pstem = PorterStemmer()

In [ ]:
def stemming(content):
  #^ means remove everything except the given i.e., a-zA-Z
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [pstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #As mentioned earlier stopwords are the words which don't have any influential meaning .
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [ ]:
#Applying the function
t_data['stemmed_content'] = t_data['text'].apply(stemming)

In [ ]:
t_data.head()


,text,id,stemmed_content
0,ï»¿text,label,text
1,I grew up (b. 1965) watching and loving the Th...,0,grew b watch love thunderbird mate school watc...
2,"When I put this movie in my DVD player, and sa...",0,put movi dvd player sat coke chip expect hope ...
3,Why do people who do not know what a particula...,0,peopl know particular time past like feel need...
4,Even though I have great interest in Biblical ...,0,even though great interest biblic movi bore de...


In [ ]:
print(t_data['stemmed_content'])

0                                                     text
1        grew b watch love thunderbird mate school watc...
2        put movi dvd player sat coke chip expect hope ...
3        peopl know particular time past like feel need...
4        even though great interest biblic movi bore de...
                               ...                        
39996    western union someth forgotten classic western...
39997    movi incred piec work explor everi nook cranni...
39998    wife watch movi plan visit sicili stromboli so...
39999    first watch flatlin amaz necessari featur good...
40000    would film good gross estim award nomin john t...
Name: stemmed_content, Length: 40001, dtype: object


In [ ]:
t_data['id'].value_counts()

0        20019
1        19981
label        1
Name: id, dtype: int64

In [ ]:
t_data = t_data.drop(t_data.index[0])

In [ ]:
# t_data = t_data.drop(label= "label", axis = 0)

TypeError: DataFrame.drop() got an unexpected keyword argument 'id'

In [ ]:
#seperating data and label
X = t_data['stemmed_content'].values
Y = t_data['id'].values

In [ ]:
print(X)

['grew b watch love thunderbird mate school watch play thunderbird school lunch school want virgil scott one want alan count becam art form took children see movi hope would get glimps love child bitterli disappoint high point snappi theme tune could compar origin score thunderbird thank earli saturday morn one televis channel still play rerun seri gerri anderson wife creat jonatha frake hand director chair version complet hopeless wast film utter rubbish cgi remak may accept replac marionett homo sapien subsp sapien huge error judgment'
 'put movi dvd player sat coke chip expect hope movi would contain strong point first movi awsom anim good flow stori excel voic cast funni comedi kick ass soundtrack disappoint found atlanti milo return read review first might let follow paragraph direct seen first movi enjoy primarili point mention br br first scene appear shock pick atlanti milo return display case local videoshop whatev expect music feel bad imit first movi voic cast replac fit one

In [ ]:
print(Y)

['0' '0' '0' ... '0' '1' '1']


Splitting data into training and testing data

In [ ]:
t_data['id'].value_counts()

0    20019
1    19981
Name: id, dtype: int64

In [ ]:
#Splitting data into training and testing data
# x_train will contain all the training data tweets.
# y_train will contain all the training data targets(LABELS).
# x_test will contain all the testing data tweets.
# y_test will contain all the testing data targets(LABELS).
#If we don't set stratify then there is the chance of unfair splitting
#regarding the random state, each time you split data, it will be split differently.
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, stratify = Y, random_state = 2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(40000,) (32000,) (8000,)


In [ ]:
#Converting the text to numerical value using feature extraction.
#Using method called Vectorizer.
#Prioritize according to frequency.
#Use the fit_transform method for training data alone not for test data.
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 16729)	0.09965441813678852
  (0, 15720)	0.1194055633672808
  (0, 55830)	0.10357369639363999
  (0, 6076)	0.18620259348517748
  (0, 50230)	0.15502323258209308
  (0, 3503)	0.17049092369098562
  (0, 15740)	0.14413597143028065
  (0, 39637)	0.13228150228255167
  (0, 14760)	0.13537425709965595
  (0, 10754)	0.07403050814947817
  (0, 11856)	0.22533299437428794
  (0, 25678)	0.1566992216745792
  (0, 7370)	0.11644783833534952
  (0, 4384)	0.09226268074946864
  (0, 43481)	0.13561869666771623
  (0, 16089)	0.0630137275246874
  (0, 23092)	0.10357369639363999
  (0, 43527)	0.07086671305208773
  (0, 51827)	0.07975653544526332
  (0, 35644)	0.046946745223642075
  (0, 33337)	0.06870781286955946
  (0, 39089)	0.09539440645564308
  (0, 9903)	0.09943074371579685
  (0, 2980)	0.20916109043153844
  (0, 35546)	0.09095048276283013
  :	:
  (31999, 33607)	0.06847317973952394
  (31999, 16143)	0.22265046499951266
  (31999, 37400)	0.061250906513190664
  (31999, 39248)	0.1552890841699979
  (31999, 29003)	0.0644514391

In [ ]:
print(X_test)

#The first element 0 means words are in 0th tweet and so on.
#The second element in the output means,

  (0, 56464)	0.14228986745285988
  (0, 55856)	0.08665342803765183
  (0, 55843)	0.03592415635747062
  (0, 55189)	0.05485207132367911
  (0, 55149)	0.17344894146412485
  (0, 54442)	0.11680137211116629
  (0, 53097)	0.16482578442575851
  (0, 51827)	0.08892770422408257
  (0, 51028)	0.1025544625724923
  (0, 50923)	0.06925381168375618
  (0, 50637)	0.06014239448366717
  (0, 50418)	0.03178876793957485
  (0, 49182)	0.061900451234796885
  (0, 48807)	0.08222197222865134
  (0, 46980)	0.22254211148163663
  (0, 46442)	0.07400868839237164
  (0, 45669)	0.13635798185398612
  (0, 45293)	0.06665833070144948
  (0, 43868)	0.05676735372593144
  (0, 43527)	0.039507822540474094
  (0, 43360)	0.06411231616605235
  (0, 43300)	0.08749680138052664
  (0, 43199)	0.3540666132379534
  (0, 42198)	0.3115705124558581
  (0, 40668)	0.03706182471813905
  :	:
  (7999, 19369)	0.07214224414339361
  (7999, 18907)	0.1345300440536637
  (7999, 18122)	0.07168999868507414
  (7999, 18069)	0.09215856317426259
  (7999, 17954)	0.378375713

Training the Logistic Regression Model.
(One of the model in the ML used for binary classification and predictive analytics)

In [ ]:
model = LogisticRegression(max_iter = 1000)

Model Training phase

In [ ]:
#Model will learn and train from here
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Accuracy Score

In [ ]:
#Accuracy score on the training data.
X_train_prediction =model.predict(X_train)
#In the above statement model will predict target on training data  given only training data without labels.
#In the below statement we will calculate the accuracy by comparing the generated labels
#with the actual labels.
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print("Accuracy Score on the training data: ", training_data_accuracy)


Accuracy Score on the training data:  0.92578125


In [ ]:
Y_test_prediction =model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,Y_test_prediction)

Here as the accuracy of training and test data is close, we can say that model has performed pretty well.

Overfitting: Training data accuracy is much more than test data accuracy.
ex. training data accuracy = 80% and test data accuracy = 40-50%


Underfitting: Model performs poorly in both training and testing data, shows that the model is too simple for the data, need to design more complex model.

In [ ]:
print(f"Accuracy Score on Test Data: {test_data_accuracy *100:.3f} %")

Accuracy Score on Test Data: 89.912 %


In [ ]:
#Saving the model so that you can use it without training later.
import pickle

filename = "modellr.sav"
#dumping the model, wb means write in binary format.
pickle.dump(model, open(filename, 'wb'))


In [ ]:
#fetching the saved file.and using the already trained model directly for prediction.
#Using the model for new predictions.
#Load the model.
load_model = pickle.load(open('/content/modellr.sav', 'rb'))

In [ ]:
#This is nothing but the 200th data point
X_new = X_test[7999]
print(Y_test[7999])

1


In [ ]:
prediction = load_model.predict(X_new)
print(*prediction)

if (prediction[0]=='0'):
  print("It is a negative review")
else:
  print("It is a positive review")

1
It is a positive review


In [ ]:
X_new = X_test[2]
print(Y_test[2])
prediction = load_model.predict(X_new)

if (prediction[0]==0):
  print("It is a negative tweet")
else:
  print("It is a positive tweet")

1
['1']
It is a positive tweet


In [ ]:
#Here our project is complete.
